# Assignment 10

In this assignment, we want to implement **cross-validation** for logistic regression. Cross-validation is a powerful technique for model selection (such as when choosing the right hyper-parameters), especially when the data size is not very large. The goal of this assignment is to first implement cross-validation and then to compare it to a baseline model (with no cross-validation).

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore") # warnings.simplefilter("default")
%matplotlib inline
random_state = 0

### Get the data
Read in `AmesFeatures.csv` which contains the processed data from the lecture.  This data file can be found in canvas.  If you want, you can generate this data file yourself by uncommenting the line `# AmesFeatures.to_csv('AmesFeatures.csv', index=False)` in file `Lesson_10_b_Student.ipynb`.  You may need to change the path below.  <br/><span style="color:red" float:right>[0 point]</span>

In [ ]:
AmesFeatures = pd.read_csv('AmesFeatures.csv')
y = (AmesFeatures['SalePrice'] > 200000) & (AmesFeatures['SalePrice'] < 230000)
X = AmesFeatures.drop(columns=['Id', 'SalePrice'])
display(y.value_counts())
display(X.shape)
display(X)

Use some of the code from the lecture. 
- Split X and y into X_train, X_test, y_train, and y_test using `test_size = 0.30`.  You may want to use `random_state = 0` to make your results the same as others 
- Present the counts for False and True in y_train and y_test to verify the imbalanced data in your train and test sets
<br/><span style="color:red" float:right>[0 point]</span>

In [ ]:
# Add code here to splt data into test and train sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = random_state)
# Add code to present the imbalance of the class labels for both test and training


1. Train basic `LogisticRegression` classifier (repeat what was done in class)
 1. Train a model with `X_train` and `y_train`
 2. predict (hard baseline and soft) on training and test features with `.predict()` and `.predict_proba()`
 3. evaluate model using confusion matrix (`confusion_matrix`) and its metrics (`classification_report`)
 4. evaluate model using ROC and AUC of ROC
 5. comment on the model's usability. 
<br/><span style="color:red" float:right>[2 point]</span>

In [ ]:
# Add code here to train the logistic regression


In [ ]:
# Add code to predict (hard and soft) on training and test features


In [ ]:
# Add code to evaluate predictions using confusion matrix and its metrics


In [ ]:
# Add code to evaluate predictions using ROC


#### Add Comment on usability


2. Train `LogisticRegression` with balanced class weights 
 1. Read the documentation to see what `class_weight` does
 2. Train a new model with the same `X_train` and `y_train` settting `class_weight` so the weights are balanced
 3. predict (hard and soft) on training and test features with .predict() and .predict_proba()
 4. evaluate model using confusion matrix (confusion_matrix) and its metrics (classification_report)
 5. evaluate model using ROC
 6. How does balancing class weights change any of the results? Why?
<br/><span style="color:red" float:right>[3 point]</span>

In [ ]:
# Add code here to train the logistic regression with balanced weights


In [ ]:
# Add code to predict (hard and soft) on training and test features


In [ ]:
# Add code to evaluate predictions using confusion matrix and its metrics


In [ ]:
# Add code to evaluate predictions using ROC


#### Add Comment on the effect of weight balancing


3. Use `LogisticRegressionCV` to train a cross-validation logistic regression.  The CV stands for cross-validation.
 1. train a cross-validation logistic regression
    - Use the same `X_train` and `y_train`.
    - You may want to use `random_state = 0` to make your results the same as others.
    - Set the `cv` parameter to 5.  5 is the default value.
    - Set `class_weight` so the weights are balanced
 2. predict (hard and soft) on training and test features with `.predict()` and `.predict_proba()`
 3. evaluate test and training predictions using confusion matrix (`confusion_matrix`) and its metrics (`classification_report`)
 4. evaluate test and training predictions using ROC
 5. comment on whether cross-validation makes a difference in the results. 
<br/><span style="color:red" float:right>[3 point]</span>

In [ ]:
# Add code to train logistic regression cross-validation (cv = 5)


In [ ]:
# Add code to predict (hard and soft) on training and test features


In [ ]:
# Add code to evaluate predictions using confusion matrix and its metrics


In [ ]:
# Add code to evaluate predictions using ROC


#### Add comments on model evaluation and  cross-validation


4. Increase the number of folds and train the CV model again:
 1. train a cross-validation logistic regression
    - Use the same `X_train` and `y_train`.
    - You may want to use `random_state = 0` to make your results the same as others.
    - Set the `cv` parameter to 10 
    - Set `class_weight` so the weights are balanced
 2. predict (hard and soft) on training and test features with `.predict()` and `.predict_proba()`
 3. evaluate test and training predictions using confusion matrix (`confusion_matrix`) and its metrics (`classification_report`)
 4. evaluate test and training predictions using ROC
 5. comment on whether cross-validation makes a difference in the results. 
<br/><span style="color:red" float:right>[2 point]</span>

In [ ]:
# Add code to train logistic regression cross-validation (cv = 10)


In [ ]:
# Add code to predict (hard and soft) on training and test features


In [ ]:
# Add code to evaluate model using confusion matrix and its metrics


In [ ]:
# Add code to evaluate model using ROC


#### Add comments on model evaluation and extended cross-validation


5. What was the cost of increasing the number of folds in terms of training run-time? <span style="color:red" float:right>[2 point]</span>

In [ ]:
# Add code here to determine cost of increasing folds


#### Add comments on training run-time here


# End of assignment